# Dispersion analysis on P wave on the unstimulated side to support field data FWI results

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import ndimage
import math
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter
from matplotlib import patches

import seppy
import os

rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# read data, mask, headers, model, source functions

sep = seppy.sep()

data_axes, data = sep.read_file(datapath+"../../ch4/dat/ch4_das.H")
pdata = data.reshape(data_axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5_data.HH")
pattr = data.reshape(axes.n,order='F').T

model_axes, data = sep.read_file(datapath+"ch5_model.H")
pmodel = data.reshape(model_axes.n,order='F').T

dt=data_axes.d[0]
dx=data_axes.d[1]
nt=data_axes.n[0]
ntr=data_axes.n[1]
ns=data_axes.n[2]

sxz=np.zeros((ns,4))
sxz[:,0]=pattr[:,0,0]
sxz[:,1]=pattr[:,0,19]*1e-5
sxz[:,2]=pattr[:,0,21]*1e-5
sxz[:,3]=pattr[:,0,4]

srxz=np.zeros((ns,ntr,6))
srxz[:,:,0]=pattr[:,:,0]
srxz[:,:,1]=pattr[:,:,19]*1e-5
srxz[:,:,2]=pattr[:,:,21]*1e-5
srxz[:,:,3]=pattr[:,:,16]*1e-5
srxz[:,:,4]=pattr[:,:,18]*1e-5
srxz[:,:,5]=pattr[:,:,24]

In [ ]:
# list of shots to drop for low quality
select=np.array([4,9,14,19,22,27,35,41,46,52,57,61,62,66,71,76,77,81,87,90,91,92,95,96,97])
pattr[:,:,28]=1
pattr[select[:].astype(int),:,28]=0

# keep the second perf in each stage
sxz1=sxz[pattr[:,0,27]==2,:]
srxz1=srxz[pattr[:,0,27]==2,:,:]

# Drop the last shot (19 shots remaining)
sxz2=sxz1[:-1]
srxz2=srxz1[:-1]

In [ ]:
extent=[model_axes.o[1],model_axes.o[1]+(model_axes.n[1]-1)*model_axes.d[1],model_axes.o[0]+(model_axes.n[0]-1)*model_axes.d[0],model_axes.o[0]]

plt.figure(figsize=(16,6))
plt.imshow(np.transpose(pmodel[0,:,:]),cmap='jet',extent=extent,vmin=2.5,vmax=6.0,interpolation='bilinear', aspect='auto')
plt.colorbar().set_label(label="km/s",size=16)
plt.scatter(sxz2[:,1],sxz2[:,2],s=50,c=sxz2[:,3],marker='x')
plt.xlabel(r'X (km)',fontsize=16)
plt.ylabel(r'Depth (km)',fontsize=16)

In [ ]:
# select shots and create groups

# group 1: second perf in each stage
pdata1=pdata[sxz1[:,0].astype(int),:,:]
pattr1=pattr[sxz1[:,0].astype(int),:,:]

# group 2: group1 without the last shot
pdata2=pdata[sxz2[:,0].astype(int),:,:]
pattr2=pattr[sxz2[:,0].astype(int),:,:]

In [ ]:
# compute the frequency-velocity spectra for limited offset range
sep.write_file("temp.H", np.transpose(pdata2), ds=np.array([dt,1,1]), os=np.array([0,-800,0]), dpath=pwd[0]+"/")

!Window < temp.H min2=-300 max2=-100 | FV.x vmin=-6500 vmax=-1500 nv=501 | Transp plane=12 | Smooth rect2=2 repeat=3 > temp.H.fv datapath=./

spec_axes, spec = sep.read_file("temp.H.fv")
pspec = spec.reshape(spec_axes.n,order='F').T

!rm -rf temp.H*

In [ ]:
sp=15
p=0.6
vmax=p*np.amax(pspec[sp])

fig=plt.figure(figsize=(3.33, 2),dpi=300)

plt.imshow(np.transpose(pspec[sp]/vmax), interpolation='bilinear', aspect='auto', extent=[0,1000,1.5,6.5],cmap='jet',vmin=0,vmax=1)
ellipse = patches.Ellipse(xy=(550, 3.9), width=250, height=0.8, edgecolor='w', fc='None', lw=1, ls='--')
plt.gca().add_patch(ellipse)
plt.xlim(0,700)
plt.ylim(1.5,6.5)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Phase velocity (km/s)')
# plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
# plt.colorbar(format="%.1f",pad=0.01,shrink=0.5)

plt.savefig(figpath+'das_dispersion.png',bbox_inches='tight',format='png')

In [ ]:
# pick the maximum in a given frequency-velocity window
fmin=500
fmax=600
ifmin = int((fmin - spec_axes.o[1]) / spec_axes.d[1])
ifmax = int((fmax - spec_axes.o[1]) / spec_axes.d[1])

vmin=-5000
vmax=-3000
ivmin = int((vmin - spec_axes.o[0]) / spec_axes.d[0])
ivmax = int((vmax - spec_axes.o[0]) / spec_axes.d[0])

indices = np.argmax(pspec[:,ifmin:ifmax,ivmin:ivmax],axis=2) # index of maximum velocity at each frequency
maxvel= (indices + ivmin)*spec_axes.d[0] + spec_axes.o[0] # maximum velocity
medvel = np.median(maxvel,axis=1) # median of maximum velocities across frequencies
meanvel = np.mean(maxvel,axis=1) # average of maximum velocities across frequencies

xcoord = srxz2[:,600,3]
array = np.array([xcoord,-0.001*medvel])

In [ ]:
# Save the velocity profile to be read in other notebooks for figures
np.save(datapath+'dispersion_velocity',array)

In [ ]:
# load the P-wave dispersion velocity profile
disperse_vel=np.load(datapath+'dispersion_velocity.npy')

In [ ]:
# figure plot for Chapter 5

fig=plt.figure(figsize=(10, 2),dpi=300)

plt.plot(disperse_vel[0],disperse_vel[1],linewidth=2, c='k',marker='x')
plt.xlim([0,2.05])
plt.xlabel('X (km)')
plt.ylabel('Velocity (km/s)')
plt.tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='k', linestyle='-', linewidth=0.2)

plt.savefig(figpath+'dispersion_velocity.png',bbox_inches='tight',format='png')